In [3]:
#########################################################################################################################
# Kurs: Data science, studia zaoczne, Informatyka w Biznesie
# Data ostatniej poprawki: 04.12.2018
# Autorzy: Mariusz Czarny, Juliusz Sadowski, Anita Koszalko

# Celem projektu jest zastosowanie języka Python przy użyciu bibliotek pandas, nympy, itp., aby wykonać analizę danych.
# Danymi źródłowymi są pliki tekstowe: campaigns.txt, customers.txt, orderlines.txt, orders.txt, products.txt pobrane z
# serwera: https://drive.google.com/drive/folders/0B1A_5ELLOtW4VXgwOG1oZmhpS0k
# Ścieżka źródłowa dla plików: powinny znajdować się w tym samym katalogu co plik projektowy
# Wersja pythona: 3.6.3
# Plik dostępny jest także pod linkiem: 
# macierz korelacji
# funckja celu
# predykcja sprzedazy
# probka testowa/uczaca
# regresja liniowa/logistyczna

# User story: 
#########################################################################################################################

In [4]:
# import wewenętrznej bazy danych
import sqlite3

In [5]:
# import standardowych bibliotek
import pandas as pd
import numpy as np
import os

In [6]:
# import bibliotek służących do wizualizacji
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [7]:
# sprawdza aktualną ścieżkę
%pwd

'C:\\Users\\student\\Desktop\\dataScience-master\\dataScience-master'

In [8]:
# połączenie z bazą i wykorzystanie kursora
dbConnection = sqlite3.connect("bookdb");
dbCursor = dbConnection.cursor()

In [9]:
# rozszerzenie plików zawierających dane do wczytania
txtExtention = ".txt"

In [20]:
# tablica z nazwami plików do oczytu, zawierające dane, które będą importowane do tabel bazodanowych
tables = ["Customers", "Orderlines", "Products", "Orders"]

In [21]:
# wczytywanie kolejnych plików w pętli for
for table in tables:
    # ustawienie scieżki (nazwy plików pisane małymy literami)
    path = os.path.expanduser(table.lower() + txtExtention)
    # wczytywanie pliku używając delimitera tabulacji oraz kodowania 'latin1'
    file = pd.read_csv(path, delimiter="\t", encoding = 'latin1')
    # tworzy na podstawie pliku tabele i wypełnia ją rekordami, jesli istnieje zostaje nadpisana
    file.to_sql(table, dbConnection, if_exists="replace");

In [22]:
import seaborn as sns
iris = sns.load_dataset('iris')

In [23]:
selectCities = '''SELECT city,numorderlines,numunits from orders '''
citiesDF = pd.read_sql_query(selectCities, dbConnection)

In [24]:
onlyThreecities=citiesDF.loc[citiesDF["city"].isin(["NEW YORK", "WASHINGTON", "BROOKLYN"])]
onlyThreecities.head()

,city,numorderlines,numunits
14,BROOKLYN,1,1
21,NEW YORK,1,3
29,NEW YORK,1,1
33,NEW YORK,1,2
48,NEW YORK,1,1


In [25]:
%matplotlib inline
sns.set()

In [26]:
sns.pairplot(onlyThreecities, hue='city', height=1.5);

TypeError: pairplot() got an unexpected keyword argument 'height'

In [27]:
# Correlation heatmap

In [28]:
# calculate the correlation matrix
corr = onlyThreecities.corr()

In [29]:
# plot the heatmap
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

ValueError: Arrays were different lengths: 2 vs 0

In [31]:
ordersSelect = '''SELECT totalprice, customerid,orderid from orders '''
orders = pd.read_sql_query(ordersSelect, dbConnection)
orders.head()

,totalprice,customerid,orderid
0,190.00,45978,1002854
1,10.00,125381,1002855
2,35.22,103122,1002856
3,10.00,130980,1002857
4,10.00,48553,1002886


In [32]:
customersSelect = '''SELECT customerid, gender from customers '''
customers = pd.read_sql_query(customersSelect, dbConnection)

In [33]:
customers.head()

,customerid,gender
0,174596,M
1,68239,M
2,10203,F
3,174288,F
4,68099,M


In [34]:
ordersCustomer = orders.merge(customers, on='customerid')

In [35]:
ordersCustomer.head()

,totalprice,customerid,orderid,gender
0,190.00,45978,1002854,M
1,10.00,125381,1002855,M
2,35.22,103122,1002856,None
3,10.00,130980,1002857,F
4,10.00,48553,1002886,None


In [37]:
orderLinesSelect = '''SELECT orderlineid, orderid, productid, billdate from Orderlines '''
orderLines = pd.read_sql_query(orderLinesSelect, dbConnection)

In [38]:
orderLines.head()

,orderlineid,orderid,productid,billdate
0,1010561,1006414,10834,2011-03-08 00:00:00
1,1010562,1006541,11052,2011-01-20 00:00:00
2,1010563,1006542,11070,2011-01-20 00:00:00
3,1010564,1010154,11196,2009-11-20 00:00:00
4,1010565,1009110,11048,2009-11-12 00:00:00


In [39]:
orderlinesOrderCustomer = orderLines.merge(ordersCustomer, on='orderid')

In [40]:
orderlinesOrderCustomer.head()

,orderlineid,orderid,productid,billdate,totalprice,customerid,gender
0,1010561,1006414,10834,2011-03-08 00:00:00,36.00,48167,M
1,1010560,1006414,10842,2011-01-20 00:00:00,36.00,48167,M
2,1010562,1006541,11052,2011-01-20 00:00:00,20.00,15824,M
3,1010563,1006542,11070,2011-01-20 00:00:00,19.12,124842,M
4,1010564,1010154,11196,2009-11-20 00:00:00,14.95,49720,M


In [46]:
orderlinesOrderCustomer["billdate"] = pd.to_datetime(orderlinesOrderCustomer["billdate"]).dt.year

In [47]:
orderlinesOrderCustomer.head()

,orderlineid,orderid,productid,billdate,totalprice,customerid,gender
0,1010561,1006414,10834,2011,36.00,48167,M
1,1010560,1006414,10842,2011,36.00,48167,M
2,1010562,1006541,11052,2011,20.00,15824,M
3,1010563,1006542,11070,2011,19.12,124842,M
4,1010564,1010154,11196,2009,14.95,49720,M


In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB

model = LinearRegression(fit_intercept=True)
model = GaussianNB()

In [53]:
pd.get_dummies(orderlinesOrderCustomer["gender"])

,0
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [51]:


model.fit(x[, np.newaxis], y)


xfit = np.linspace(0, 10, 1000)
yfit = model.predict(xfit)

plt.scatter(x, y)
plt.plot(xfit, yfit);

SyntaxError: invalid syntax (<ipython-input-51-3017d6c122ef>, line 3)